In [ ]:
import pandas as pd
import numpy as np
import os
import typing
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

df = pd.read_csv('data/K120.csv', delimiter=';').to_numpy()
data = np.array(df[:, 1:], dtype=float)

In [ ]:
#https://colab.research.google.com/drive/1b3CUJuDOmPmNdZFH3LQDmt5F0K3FZhqD?usp=sharing#scrollTo=bY2yEu2QTBXP
def df_to_X_y(data, window_size=5):
  X = []
  y = []
  for i in range(len(data)-window_size):
    row = [[a] for a in data[i:i+window_size]]
    X.append(row)
    label = data[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

#split and normalize the data
window_size = 10
X, Y = df_to_X_y(data, window_size=window_size)
X.shape

In [ ]:
num_of_steps = data.shape[0]
train_size = 0.5
val_size = 0.2

num_train = int(num_of_steps * train_size)
num_val = int(num_of_steps * val_size)

train_set = data[:num_train]
mean, std = np.nanmean(train_set, axis=0), np.nanstd(train_set, axis=0)

train_set = (train_set - mean) / std
val_set = (data[num_train:num_train + num_val] - mean) / std
test_set = (data[num_train + num_val:] - mean) / std

print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

In [ ]:
from keras.utils import timeseries_dataset_from_array

#inputs series of sequential inputs


#targets, depending on the forecast horizon and if we want to return multiples a

batch_size = 32
input_sequence_length = 8
forecast_horizon = 1
multi_horizon = False

def create_tf_dataset(
    data_array: np.ndarray,
    input_sequence_length: int,
    forecast_horizon: int,
    batch_size: int = 128,
    shuffle=True,
    multi_horizon=False,
):
    inputs = timeseries_dataset_from_array(
        np.expand_dims(data_array[:-forecast_horizon], axis=-1),
        None,
        sequence_length=input_sequence_length,
        shuffle=False,
        batch_size=batch_size,
    )

    target_offset = (
        input_sequence_length
        if multi_horizon
        else input_sequence_length + forecast_horizon - 1
    )
    target_seq_length = forecast_horizon if multi_horizon else 1
    targets = timeseries_dataset_from_array(
        data_array[target_offset:],
        None,
        sequence_length=target_seq_length,
        shuffle=False,
        batch_size=batch_size,
    )

    dataset = tf.data.Dataset.zip((inputs, targets))
    if shuffle:
        dataset = dataset.shuffle(100)

    return dataset.prefetch(16).cache()

train_dataset, val_dataset = (
    create_tf_dataset(data_array, input_sequence_length, forecast_horizon, batch_size)
    for data_array in [train_set, val_set]
)

test_dataset = create_tf_dataset(
    test_set,
    input_sequence_length,
    forecast_horizon,
    batch_size=test_set.shape[0],
    shuffle=False,
    multi_horizon=multi_horizon,
)


In [ ]:
list(train_dataset.as_numpy_iterator())

In [ ]:
import keras.layers

batch_size = 32
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = data.shape[1]

units = 64
output_size = data.shape[1]  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [ ]:
from keras.losses import MeanSquaredError
from keras.optimizers import Adam
model = build_model()
model.summary()
model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)

model.fit(
    x=train_dataset,
    validation_data=val_dataset,
    epochs=500,
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)